In [1]:
import torch
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision.models import ResNet50_Weights
import numpy as np
import torchvision.transforms as transforms        
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from src.temperature_scaling import ModelWithTemperature
from src.raps_hyp_opt import lambda_optimization_raps, k_reg_optimization

#  Reprocess
data_transform = transforms.Compose([
    transforms.CenterCrop(256),
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

sorted_val_path = "D:\\Download\\ImageNet-1K\\Validation_Set\\sorted_ImageNet_val"
dataset = ImageFolder(root=sorted_val_path, transform=data_transform)

# load pre-trained model 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1).to(device)

subset_size = len(dataset) // 10
indices = np.random.choice(len(dataset), subset_size, replace=False)
subset_dataset = Subset(dataset, indices)
train_loader = DataLoader(subset_dataset, batch_size=32, shuffle=False, num_workers=2)
model = ModelWithTemperature(model, temperature = 1.0).to(device)
model.set_temperature(train_loader)

model.eval()

lambda_values = [0, 1e-4, 1e-3, 0.01, 0.02, 0.1, 0.05, 0.2, 0.5, 1.0]
k_reg_values = [1, 2, 3, 4, 5, 10, 50]

# lambda optimization
print("Looking for optimal lambda...")
optimal_lambda = lambda_optimization_raps(model, dataset, lambda_values, k_reg = 5, device=device)
if optimal_lambda is None:
    print("No optimal lambda is found")
else:
    print(f"Optimal lambda is {optimal_lambda}\n")
    
    # k_reg optimization
    print("Looking for optimal k_reg...")
    optimal_k = k_reg_optimization(model, dataset, optimal_lambda, k_reg_values, device=device)
    
    if optimal_k is None:
        print("No optimal k_reg is found")
    else:
        print(f"Optimal k_reg is {optimal_k}")

Before temperature - NLL: 1.132, ECE: 0.021
Optimal temperature: 0.984
After temperature - NLL: 1.131, ECE: 0.024
Looking for optimal lambda...
Optimal lambda is 0.1

Looking for optimal k_reg...
Optimal k_reg is 4


# Result

- Optimal lambda: **0.1**
- Optimal k_reg: **4**